In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#get all csv files as pandas dataframes
anime = pd.read_csv('/kaggle/input/anime-recommendations-database/anime.csv')
rating = pd.read_csv('/kaggle/input/anime-recommendations-database/rating.csv')

In [ ]:
#Drop columns which may have little or no use
anime = anime.drop(['type','episodes','members'],axis=1)
anime

In [ ]:
#I would prefer indivual ratings over generalised ones
anime = anime.drop(['rating'],axis=1)
anime

In [ ]:
df = pd.merge(anime,rating)
df = df.replace(-1,0)

In [ ]:
df['user_id'].value_counts().shape 

In [ ]:
#number of ratings of each anime
number_rating = df.groupby('name')['rating'].count().reset_index()

In [ ]:
number_rating.rename(columns={'rating':'number of rating'},inplace=True)

In [ ]:
number_rating.head()

In [ ]:
df=df.merge(number_rating,on='name')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
#Drop insignifanct ratings,I have taken it as 200 you may take less or more,but the most common for these kind of datasets is 50
df=df[df['number of rating']>=200]

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.drop(columns=['number of rating'],inplace=True)

In [ ]:
df.head()

In [ ]:
df = df.replace('None',0)

In [ ]:
df.to_csv('anime.csv',index=False)

In [ ]:
#Make a pivot table for more clarity and for making a matrix
movie_pivot=df.pivot_table(columns='user_id',index='name',values='rating')

In [ ]:
movie_pivot.shape

In [ ]:
movie_pivot.fillna(0,inplace=True)

In [ ]:
pd.set_option('display.max_rows', None)
movie_pivot

In [ ]:
df.drop(columns=['user_id','rating'],inplace=True)

In [ ]:
#Make a matrix
from scipy.sparse import csr_matrix
movie_sparse=csr_matrix(movie_pivot)


In [ ]:
#The algorithm
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors( n_neighbors=10,algorithm='brute',metric='cosine')

In [ ]:
model.fit(movie_sparse)

In [ ]:
df1=df.copy()


In [ ]:
distances,suggestions=model.kneighbors(movie_pivot.iloc[1000,:].values.reshape(1,-1))

In [ ]:
movie_pivot.iloc[1000,:].values.reshape(1,-1)

In [ ]:
distances

In [ ]:
suggestions

In [ ]:
for i in range(len(suggestions)):
    print(movie_pivot.index[suggestions[i]])

In [ ]:
#Define a function to recommend animes
def reco(anime_name):
    anime_id=movie_pivot.index.get_loc(anime_name)
    distances,suggestions=model.kneighbors(movie_pivot.iloc[anime_id,:].values.reshape(1,-1))

    
    
    
    for i in range(len(suggestions)):
        return (movie_pivot.index[suggestions[i]])



In [ ]:
#I would keep user input, but kaggle does not support that
inp = '91 Days' 

In [ ]:
res=reco(inp)

In [ ]:
#Here are your results! Now rejoice!
for i in res:
    print(i)

If you found this notebook useful, please do upvote